In [1]:
# Random forest model for EMA crossover identifier
import pandas as pd
import numpy as np
from pathlib import Path
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
csv_path = Path('../Resources/ts_emax_x_squeeze.csv')
csv_path

PosixPath('../Resources/ts_emax_x_squeeze.csv')

In [3]:
ts_file_df = pd.read_csv(csv_path, parse_dates=[['Date', 'Time']])
ts_file_df.tail()

,Date_Time,Open,High,Low,Close,Vol,OI,UpperBand,LowerBand,MidLine,UpperBand.1,LowerBand.1,MidLine.1,FastEMA,SlowEMA,squeeze,emax_signal
19600,2021-01-12 14:00:00,140.46,140.58,140.34,140.46,152792,140537,142.13,139.93,141.03,141.69,140.37,141.03,140.76,140.44,1.0,0.0
19601,2021-01-12 14:15:00,140.47,140.92,140.47,140.60,170378,174631,142.04,139.98,141.01,141.69,140.33,141.01,140.73,140.46,1.0,0.0
19602,2021-01-12 14:30:00,140.61,140.75,140.42,140.48,177439,178475,141.98,139.99,140.99,141.71,140.27,140.99,140.68,140.46,1.0,0.0
19603,2021-01-12 14:45:00,140.47,140.50,140.10,140.17,205777,182808,141.92,139.99,140.95,141.76,140.15,140.95,140.58,140.43,1.0,0.0
19604,2021-01-12 15:00:00,140.18,140.63,140.14,140.21,502297,446696,141.81,139.98,140.89,141.72,140.06,140.89,140.51,140.41,1.0,0.0


In [4]:
# set index as Date_Time and drop MidLine.1 column (it is a duplicate of MidLine)
ts_file_df.set_index(pd.to_datetime(ts_file_df['Date_Time'], infer_datetime_format=True), inplace=True)
ts_file_df.drop(columns=['Date_Time', 'MidLine.1'], inplace=True)
ts_file_df.head()

,Open,High,Low,Close,Vol,OI,UpperBand,LowerBand,MidLine,UpperBand.1,LowerBand.1,FastEMA,SlowEMA,squeeze,emax_signal
Date_Time,,,,,,,,,,,,,,,
2018-01-12 08:45:00,111.65,111.98,110.84,111.22,1051611,1206429,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0
2018-01-12 09:00:00,111.21,112.04,111.17,111.93,645133,604771,0.0,0.0,0.0,0.0,0.0,111.93,111.93,0.0,0.0
2018-01-12 09:15:00,111.93,112.20,111.87,112.13,541130,492175,0.0,0.0,0.0,0.0,0.0,111.97,111.95,0.0,0.0
2018-01-12 09:30:00,112.13,112.18,111.56,111.91,332412,418068,0.0,0.0,0.0,0.0,0.0,111.96,111.94,0.0,0.0
2018-01-12 09:45:00,111.91,112.24,111.85,111.97,281205,290538,0.0,0.0,0.0,0.0,0.0,111.96,111.95,0.0,0.0


In [5]:
# Set a variable list of features to feed to our model

x_var_list = ['Open', 'High', 'Low', 'Close', 'Up', 'Down', 'kcup', 'kclo', 'MidLine', 'bbup', 'bblo', 'FastEMA', 'SlowEMA']
ts_file_df[x_var_list].head()

KeyError: "['Up', 'kcup', 'bblo', 'bbup', 'kclo', 'Down'] not in index"